## 3.2 LR-Zerlegung

**Implementierung 3.1: Rückwärtseinsetzen**

Wir implementieren den Algorithmus des Rückwärtseinsetzens mithilfe von `numpy`. Dabei müssen wir bei den Indizes vorsichtig sein. Auf dem Papier werden in der Regel die Indizes der Einträge einer $n$x$n$-Matrix von $1$ bis $n$ durchnummeriert, in dem meisten Programmiersprachen laufen die Indizes aber von $0$ bis $n-1$.

In [ ]:
import numpy as np

def rueckwaerts_einsetzen(R, b):
    assert (len(b.shape) == 1), 'Rechte Seite ist kein Vektor'
    n = b.shape[0]
    assert (R.shape == (n, n)), 'Matrix hat falsche Dimensionen'
    
    x = np.empty_like(b)
    
    x[n - 1] = b[n - 1] / R[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        xr = 0
        for j in range(i + 1, n):
            xr += R[i, j] * x[j]
        x[i] = (b[i] - xr) / R[i, i]
    return x   

*Ergänzende Einzelheiten zum Code*
- Am Anfang der Funktion testen wir, ob die übergebenen Daten überhaupt kompatibel sind, also ob wirklich ein Vektor übergeben wurde und ob die Matrix die richtigen Dimensionen hat.
- Mit der numpy Funktion `np.empty_like()` erstellen wir ein neues, leeres `array` mit den selben Eigenschaften wie `b`. Das heißt die Dimensionen und die Genauigkeit mit der Gleitkommazahlen gespeichert werden sind dieselben.

Wir testen unseren Code nun anhand des Beispiels
$$
R = \begin{pmatrix} 1 & 1 & 1 \\ 0 & 1 & 2 \\ 0 & 0 & 4 \end{pmatrix}
\quad\text{und}\quad
b = \begin{pmatrix} 1 \\ 1 \\ -4\end{pmatrix}.
$$

In [ ]:
R = np.array([[1, 1, 1], [0, 1, 2], [0, 0, 4]], dtype=np.float64)
b = np.array([1, 1, -4], dtype=np.float64)

Hier haben wir spezifiziert, dass die Matrixeinträge als 64-bit Gleitkommazahl gespeichert werden sollen (sonnst auch als `double` bekannt). Dies ist wichtig, da `numpy` sonst das Zahlenformat aus den Einträgen folgern würde, was in diesem Fall `int64` wäre, also ganze Zahlen.

In [ ]:
x = rueckwaerts_einsetzen(R, b)
print(' x = ', x)
print('Rx = ', np.dot(R, x))

Experimentieren Sie mit den Einträgen der Matrix und dem Zahlenformat. Z.B. nehmen Sie die rechte Seite $(1,1,-1)$ und speichern `R, b` im Datenformat `np.int64`. Was beobachten Sie und können Sie dies erklären?

**Implementierung 3.2: LR-Zerlegung einer Matrix A ohne Zusatzspeicher**

In [ ]:
def LR_zerlegung_einfach(A):
    assert (A.shape[0] == A.shape[1]), 'Matrix ist nicht quadratisch'
    n = A.shape[0]
    
    for i in range(0, n):
        for k in range(i + 1, n):
            A[k, i] = A[k, i] / A[i, i]
            for j in range(i + 1, n):
                A[k, j] = A[k, j] - A[k, i] * A[i, j]
    return None

#### Beispiel 3.13 (LR-Zerlegung ohne Pivotisierung)

Wir betrachten das Beispiel
$$
A = \begin{pmatrix}
2.3 & 1.8 & 1 \\ 1.4 & 1.1 & -0.7 \\ 0.8 & 4.3 & 2.1
\end{pmatrix}
\quad\text{und}\quad
b = \begin{pmatrix} 1.2 \\ -2.1 \\ 0.6 \end{pmatrix}.
$$

In [ ]:
A = np.array([[2.3, 1.8, 1], [1.4, 1.1, -0.7], [0.8, 4.3, 2.1]])
b = np.array([1.2, -2.1, 0.6])

Hier ist es nicht notwendig das Zahlenformat anzugeben, da die `array`s direkt mit Gleitkommazahlen gefüllt werden, die `numpy` direkt als `float64` interpretiert. Dies können Sie überprüfen in dem Sie den Datentyp durch das Attribut `dtype` der Arrays sich ausgeben lassen.

Mit dem Modul `linalg` von `numpy` können wir das System direkt lösen lassen:

In [ ]:
x_np = np.linalg.solve(A, b)
print('x_np = ', x_np)

Wir legen nun die Matrix nochmal neu mit *half precision* Dezimalzahlen an, um den Effekt der Pivotisierung zu verdeutlichen

In [ ]:
A = np.array([[2.3, 1.8, 1],[1.4, 1.1, -0.7],[0.8, 4.3, 2.1]], dtype=np.half)
b = np.array([1.2, -2.1, 0.6], dtype=np.half)

LR_zerlegung_einfach(A)

print('Modifiziertes A =\n', A)

Um das System zu lösen, müssen wir noch das Vorwärtseinsetzen implementieren:

**Implementierung 3.3: Vorwärtseinsetzen**

In [ ]:
def vorwaerts_einsetzen_ohne_diag(L, b):
    # Wir nehmen an das alle Diagonaleinträge 1 sind um die modifizierte
    # Matrix ohne extra speicher verwenden zu können
    x = np.zeros_like(b)

    for i in range(0, b.shape[0]):
        xr = 0
        for j in range(0, i):
            xr += L[i, j] * x[j]
        x[i] = (b[i] - xr)
    return x 

In [ ]:
y = vorwaerts_einsetzen_ohne_diag(A, b)
print('y = ', y)
x = rueckwaerts_einsetzen(A, y)
print('x = ', x)

Und wir haben den relativen Fehler

In [ ]:
print('Relativer Fehler:', np.linalg.norm(x - x_np) / np.linalg.norm(x_np))

#### Beispiel 3.14 (LR-Zerlegung mit Pivotisierung)

Wir betrachten dasselbe Beispiel aber nun mit der LR-Zerlegung mit Pivotsuche.

**Implementierung 3.4: LR-Zerlegung mit Pivotisierung**

In [ ]:
def LR_zerlegung_mit_pivot(A):
    assert (A.shape[0] == A.shape[1]), 'Matrix ist nicht Quadratisch'
    n = A.shape[0]
    pivot = []
    
    for i in range(0, n):
        # Wir suchen das Pivotelement und vertauschen die Zeilen.
        k = i
        for j in range(i, n):
            if abs(A[j, i]) > abs(A[k, i]):
                k = j
        A[[i, k], :] = A[[k, i], :]
        pivot.append([i, k])
        
        for k in range(i + 1, n):
            A[k, i] = A[k, i] / A[i, i]
            for j in range(i + 1, n):
                A[k, j] = A[k, j] - A[k, i] * A[i, j]

    return pivot

Wenden wir dies auf das Gleichungssystem an:

In [ ]:
A = np.array([[2.3, 1.8, 1], [1.4, 1.1, -0.7], [0.8, 4.3, 2.1]], dtype=np.half)
b = np.array([1.2, -2.1, 0.6], dtype=np.half)

pivot = LR_zerlegung_mit_pivot(A)
print('Modifiziertes A =\n', A)

In [ ]:
for p in pivot:
    b[p] = b[[p[1], p[0]]] 
print('P b = ', b)

In [ ]:
y = vorwaerts_einsetzen_ohne_diag(A, b)
print('y = ', y)
x = rueckwaerts_einsetzen(A, y)
print('x = ', x)

Der relative Fehler ist damit

In [ ]:
np.linalg.norm(x - x_np) / np.linalg.norm(x_np)

Ändern Sie den Datentyp der Arrays zu `np.single`oder `np.double`. Was beobachten Sie? 